# Using Fairlearn with Census Data

This notebook shows how to use `fairlearn` and the Fairness dashboard to generate models for the Census dataset. This dataset is a classification problem - given a range of data about 32,000 individuals, predict whether their annual income is above or below fifty thousand dollars per year.

Income data are well known to have biases; in this case, we will attempt to mitigate the bias based on sex.

## Load and Preprocess the Dataset

For simplicity, we import the dataset from the `shap` package, which contains the data in a cleaned format. We start by importing the various modules we're going to use:

In [ ]:
import sys
sys.path.insert(0, "../")

from fairlearn.metrics import DemographicParity
from fairlearn.reductions import GridSearch
from fairlearn.reductions.grid_search.simple_quality_metrics import SimpleClassificationQualityMetric
from fairlearn.moments import MisclassificationError, DP

from sklearn import svm, neighbors, tree
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.linear_model import LogisticRegression
import pandas as pd
import shap

import numpy as np

print(sys.version)

shap.initjs()

We can now load and inspect the data from the `shap` package:

In [ ]:
X_raw,y = shap.datasets.adult()
X_raw

We are going to treat the sex of each individual as a protected attribute, and in this particular case we are going separate it out and drop it from the main data. We then perform some standard data preprocessing steps to convert the data into a format suitable for the ML algorithms

In [ ]:
A = X_raw["Sex"]
X = X_raw.drop(labels=['Sex'],axis = 1)
X = pd.get_dummies(X)

sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

le = LabelEncoder()
y = le.fit_transform(y)

Finally, we now perform the normal split of the data into training and test sets:

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(X_scaled, 
                                                    y, 
                                                    A,
                                                    test_size = 0.2,
                                                    random_state=0,
                                                    stratify=y)

# Work around indexing bug
x_train = x_train.reset_index(drop=True)
a_train = a_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
a_test = a_test.reset_index(drop=True)

## Training an unmitigated model

To show the effect of `fairlearn` we will first train a model without it. For speed of demonstration, we use a simple logistic regression learner from `sklearn`:

In [ ]:
unmitigated_model = LogisticRegression(solver='liblinear', fit_intercept=True)

unmitigated_model.fit(x_train, y_train)

We can load this model into the Fairness dashboard, and examine how it is unfair (there is a warning about AzureML since we are not yet integrated with that product):

In [ ]:
from azureml.contrib.explain.model.visualize import FairnessDashboard

FairnessDashboard(unmitigated_model, x_test, y_test.tolist(), pd.DataFrame(a_test).values.tolist(), True, list(x_test.columns), [0, 1], ["Sex"])

I'm pretty sure that something can be seen, but the widget isn't co-operating for me right now

## Mitigation with GridSearch

The `GridSearch` class in `fairlearn` implements a simplified version of the exponentiated gradient algorithm of ([Agarwal et al. 2018](https://arxiv.org/abs/1803.02453)). By restricting the problem domain to binary classifiers with binary protected attributes, the exponentiated gradient algorithm can be reduced to trying a sequence of reweightings and relabellings of the training data. This sequence is parameterised by $\lambda$ since in the full algorithm, the sweep comes from a Lagrange multiplier.

To work, `GridSearch` requires an underlying learner (we shall use the same `LogisticRegression` learner as above), a fairness metric, and a quality metric.

For this example, we specify demographic parity (on the protected attribute of "sex") as the fairness metric. This is for simplicity; in general, the appropriate fairness metric will depend on the problem specification.

The particular quality metric is not relevant in this case, since we are going to inspect a collection of models. It is used to select one of the models for the `GridSearch.fit()` method 

In [ ]:
sweep = GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                   fairness_metric=DemographicParity(),
                   quality_metric=SimpleClassificationQualityMetric())

Our algorithms provide `fit()` and `predict()` methods, so they behave in a similar manner to other ML packages in Python. We do however have to specify two extra arguments to `fit()` - the column of protected attribute labels, and also the number of values we wish to use for the Lagrange multiplier. The grid search will call the underlying learner once for each value, making it an ideal point to integrate with AzureML and leverage large scale execution in the cloud.

After `fit()` completes, we extract the full set of models from the `GridSearch` object.

In [ ]:
sweep.fit(x_train, y_train,
          protected_attribute=a_train,
          number_of_lagrange_multipliers=71)

models = [ x["model"] for x in sweep.all_models]

We could load these models into the Fairness dashboard now. However, the plot would be somewhat confusing due to their number. In this case, we are going to remove the models which are dominated in the accuracy-parity space by others from the sweep (note that the parity will only be calculated for the protected attribute; other potentially protected attributes will not be mitigated). In general, one might not want to do this, since there may be other considerations beyond the strict maximisation of accuracy and parity (of the given protected attribute).

In [ ]:
errors, disparities = [], []
for m in models:
    classifier = lambda X: m.predict(X)
    
    error = MisclassificationError()
    error.init(x_test, a_test, pd.Series(y_test))
    disparity = DP()
    disparity.init(x_test, a_test, pd.Series(y_test))
    
    errors.append(error.gamma(classifier)[0])
    disparities.append(disparity.gamma(classifier).max())
    
all_results = pd.DataFrame( {"model": m, "error": errors, "disparity": disparities})
non_dominated = []
for row in all_results.itertuples():
    errors_for_lower_or_eq_disparity = all_results["error"][all_results["disparity"]<row.disparity]
    if row.error <= errors_for_lower_or_eq_disparity.min():
        non_dominated.append(row)

Finally, we can put the non-dominated into the Fairness dashboard. We also add in the original, unmitigated model - it will be the one highlighted when the dashboard first loads.

In [ ]:
dashboard_models = [unmitigated_model]
dashboard_models.append([x.model for x in non_dominated])

FairnessDashboard(models, x_test, y_test.tolist(), pd.DataFrame(a_test).values.tolist(), True, list(x_test.columns), [0, 1], ["Sex"])

From this you can.... play around and see things?